In [1]:
import numpy as np
from scipy.stats import t
import pandas as pd
import matplotlib.pyplot as plt

## Simulation System 1

This system has patients with different CTAS levels (1 to 5) flowing into different departments (Triage, ED, and Inpatient). Along with this, patients undergo diagnostic tests at the Triage and ED levels. 

The simulation parameters are as follows: \
No. doctors: 100 \
No. nurses: 100 \
No. of admin staff: 70 \
No. of consultants: 10 
No. of ECG machines = 5 \
No. of CT machines = 5 \
No. of X-Ray machines = 5 


The simulation is run for 43800 minutes (i.e. one month) and LOS data from first 500 patients is discarded as warm-up data. In total 48k LOS data values are selected from each simulation run.

The seeds used for 5 rounds are: \
Round 1: 100 \
Round 2: 121 \
Round 3: 146 \
Round 4: 258 \
Round 5: 41

In [ ]:
# Read simulation results as dataframe
df1 = pd.read_csv("../results/simulation_results_system_1_1.csv", sep=r",(?![^\[]*\])")
df2 = pd.read_csv("../results/simulation_results_system_1_2.csv", sep=r",(?![^\[]*\])")
df3 = pd.read_csv("../results/simulation_results_system_1_3.csv", sep=r",(?![^\[]*\])")
df4 = pd.read_csv("../results/simulation_results_system_1_4.csv", sep=r",(?![^\[]*\])")
df5 = pd.read_csv("../results/simulation_results_system_1_5.csv", sep=r",(?![^\[]*\])")
df_list = [df1, df2, df3, df4, df5]

In [ ]:
for df in df_list:
    df.columns = df.columns.str.lstrip()

In [ ]:
df1[:5]

In [ ]:
# Read LOS data from dataframe df
# LOS values of only treated patients is considered
# Treated patients are the ones that complete their run (arrival to departure) in the ER

los_vals_1 = []
los_vals_2 = []
los_vals_3 = []
los_vals_4 = []
los_vals_5 = []

for items in df1["LOS"]:
    if items > 0:
        los_vals_1.append(items/60)

for items in df2["LOS"]:
    if items > 0:
        los_vals_2.append(items/60)
        
for items in df3["LOS"]:
    if items > 0:
        los_vals_3.append(items/60)
        
for items in df4["LOS"]:
    if items > 0:
        los_vals_4.append(items/60)
        
for items in df5["LOS"]:
    if items > 0:
        los_vals_5.append(items/60)

los_vals_list = [los_vals_1, los_vals_2, los_vals_3, los_vals_4, los_vals_5]
print(min(los_vals_1), max(los_vals_1))

In [ ]:
mean_list = []
std_dev_list = []
n_list = []

for los_vals in los_vals_list:
    los_vals = np.array(los_vals)
    mean_list.append(np.mean(los_vals))
    std_dev_list.append(np.std(los_vals))

    n_list.append(len(los_vals))

print(mean_list, std_dev_list, n_list)

In [ ]:
# Check critical values from table/ppf function
alpha = 0.05
t_critical_list = []

for n in n_list:
    dof = n - 1
    t_critical_list.append(t.ppf(1-alpha/2, dof))

print(t_critical_list)

In [ ]:
# Calculate confidence interval
# H_val = t_critical * std_dev * (np.sqrt(1 + (1/n)))

H_val_list = []
lower_bound = []
upper_bound = []

for index, t_critical in enumerate(t_critical_list):
    H_val = t_critical * std_dev_list[index] * (1/np.sqrt(n_list[index]))
    H_val_list.append(H_val)
    lower_bound.append(mean_list[index] - H_val)
    upper_bound.append(mean_list[index] + H_val)

print(lower_bound, upper_bound, H_val_list)
# print([mean - lower_bound], [upper_bound - mean])

In [ ]:
# Across rep data
var_list = [a**2 for a in std_dev_list]

a_mean = sum(mean_list)/5

a_var = 0
for means in mean_list:
    a_var += (means - a_mean) ** 2

a_var = a_var/4
a_std_dev = np.sqrt(a_var)

a_H_val = t_critical_list[0] * a_std_dev * (1/np.sqrt(5))

print(a_mean, a_std_dev, a_H_val, var_list, a_var)

In [ ]:
a_lower_bound = a_mean - a_H_val
a_upper_bound = a_mean + a_H_val
print(a_lower_bound, a_upper_bound)

In [ ]:
# Plot boxplot for confidence intervals
# fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 8))
interval_list = []
for index, mean in enumerate(mean_list):
    interval_list.append([lower_bound[index], mean, upper_bound[index]])

for index, interval in enumerate(interval_list):
    plt.boxplot(interval, medianprops=dict(color="red"), positions=[index+1])
    # plt.plot((index, index, index), interval)
    # plt.axhline(y=interval[1], color='orange', linestyle='--')
    
plt.show()

In [ ]:
# Drawing bar graph with error bars to show confidence interval

# fig, ax = plt.subplots()
# ax.bar(0, los_vals, yerr=[[mean - lower_bound], [upper_bound - mean]], align='center', alpha=0.5, ecolor='black', capsize=10)

# ax.set_ylabel('Mean LOS')
# ax.set_xticks([0])
# ax.set_xticklabels(['Hospital A'])
# ax.set_title('Mean LOS')

# plt.show()

## Simulation System 2

This system has patients with different CTAS levels (1 to 5) flowing into different departments (Triage, ED, and Inpatient). Along with this, patients undergo diagnostic tests at the Triage and ED levels. 

The simulation parameters are as follows: \
No. doctors: 50 \
No. nurses: 50 \
No. of admin staff: 35 \
No. of consultants: 5 \
No. of ECG machines = 5 \
No. of CT machines = 5 \
No. of X-Ray machines = 5 


The simulation is run for 43800 minutes (i.e. one month) and LOS data from first 500 patients is discarded as warm-up data. In total 48k LOS data values are selected from each simulation run.

The seeds used for 5 rounds are: \
Round 1: 100 \
Round 2: 121 \
Round 3: 146 \
Round 4: 258 \
Round 5: 41

In [ ]:
# Read simulation results as dataframe
df21 = pd.read_csv("../results/simulation_results_system_2_1.csv", sep=r",(?![^\[]*\])")
df22 = pd.read_csv("../results/simulation_results_system_2_2.csv", sep=r",(?![^\[]*\])")
df23 = pd.read_csv("../results/simulation_results_system_2_3.csv", sep=r",(?![^\[]*\])")
df24 = pd.read_csv("../results/simulation_results_system_2_4.csv", sep=r",(?![^\[]*\])")
df25 = pd.read_csv("../results/simulation_results_system_2_5.csv", sep=r",(?![^\[]*\])")
df_list = [df21, df22, df23, df24, df25]

In [ ]:
for df in df_list:
    df.columns = df.columns.str.lstrip()

In [ ]:
# Read LOS data from dataframe df
# LOS values of only treated patients is considered
# Treated patients are the ones that complete their run (arrival to departure) in the ER

los_vals_1 = []
los_vals_2 = []
los_vals_3 = []
los_vals_4 = []
los_vals_5 = []

for items in df21["LOS"]:
    if items > 0:
        los_vals_1.append(items/60)

for items in df22["LOS"]:
    if items > 0:
        los_vals_2.append(items/60)
        
for items in df23["LOS"]:
    if items > 0:
        los_vals_3.append(items/60)
        
for items in df24["LOS"]:
    if items > 0:
        los_vals_4.append(items/60)
        
for items in df25["LOS"]:
    if items > 0:
        los_vals_5.append(items/60)

los_vals_list = [los_vals_1, los_vals_2, los_vals_3, los_vals_4, los_vals_5]
print(min(los_vals_1), max(los_vals_1))

In [ ]:
mean_list = []
std_dev_list = []
n_list = []

for los_vals in los_vals_list:
    los_vals = np.array(los_vals[100:])
    mean_list.append(np.mean(los_vals))
    std_dev_list.append(np.std(los_vals))

    n_list.append(len(los_vals))

print(mean_list, std_dev_list, n_list)

In [ ]:
# Check critical values from table/ppf function
alpha = 0.05
t_critical_list = []

for n in n_list:
    dof = n - 1
    t_critical_list.append(t.ppf(1-alpha/2, dof))

print(t_critical_list)

In [ ]:
# Calculate confidence interval
# H_val = t_critical * std_dev * (np.sqrt(1 + (1/n)))

H_val_list = []
lower_bound = []
upper_bound = []

for index, t_critical in enumerate(t_critical_list):
    H_val = t_critical * std_dev_list[index] * (1/np.sqrt(n_list[index]))
    H_val_list.append(H_val)
    lower_bound.append(mean_list[index] - H_val)
    upper_bound.append(mean_list[index] + H_val)

print(lower_bound, upper_bound, H_val_list)
# print([mean - lower_bound], [upper_bound - mean])

In [ ]:
# Across rep data
var_list = [a**2 for a in std_dev_list]

a_mean = sum(mean_list)/5

a_var = 0
for means in mean_list:
    a_var += (means - a_mean) ** 2

a_var = a_var/4
a_std_dev = np.sqrt(a_var)

a_H_val = t_critical_list[0] * a_std_dev * (1/np.sqrt(5))

print(a_mean, a_std_dev, a_H_val, var_list, a_var, a_std_dev)

In [ ]:
a_lower_bound = a_mean - a_H_val
a_upper_bound = a_mean + a_H_val
print(a_lower_bound, a_upper_bound)

In [ ]:
# Plot boxplot for confidence intervals
# fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 8))
interval_list = []
for index, mean in enumerate(mean_list):
    interval_list.append([lower_bound[index], mean, upper_bound[index]])

for index, interval in enumerate(interval_list):
    plt.boxplot(interval, medianprops=dict(color="red"), positions=[index+1])
    # plt.plot((index, index, index), interval)
    # plt.axhline(y=interval[1], color='orange', linestyle='--')
    
plt.show()

## Simulation System 3

In [ ]:
# Read simulation results as dataframe
df31 = pd.read_csv("../results/simulation_results_system_3_1.csv", sep=r",(?![^\[]*\])")
df32 = pd.read_csv("../results/simulation_results_system_3_2.csv", sep=r",(?![^\[]*\])")
df33 = pd.read_csv("../results/simulation_results_system_3_3.csv", sep=r",(?![^\[]*\])")
df34 = pd.read_csv("../results/simulation_results_system_3_4.csv", sep=r",(?![^\[]*\])")
df35 = pd.read_csv("../results/simulation_results_system_3_5.csv", sep=r",(?![^\[]*\])")
df_list = [df31, df32, df33, df34, df35]

In [ ]:
for df in df_list:
    df.columns = df.columns.str.lstrip()

In [ ]:
# Read LOS data from dataframe df
# LOS values of only treated patients is considered
# Treated patients are the ones that complete their run (arrival to departure) in the ER

los_vals_1 = []
los_vals_2 = []
los_vals_3 = []
los_vals_4 = []
los_vals_5 = []

for items in df31["LOS"]:
    if items > 0:
        los_vals_1.append(items/60)

for items in df32["LOS"]:
    if items > 0:
        los_vals_2.append(items/60)
        
for items in df33["LOS"]:
    if items > 0:
        los_vals_3.append(items/60)
        
for items in df34["LOS"]:
    if items > 0:
        los_vals_4.append(items/60)
        
for items in df35["LOS"]:
    if items > 0:
        los_vals_5.append(items/60)

los_vals_list = [los_vals_1, los_vals_2, los_vals_3, los_vals_4, los_vals_5]
print(min(los_vals_1), max(los_vals_1))

In [ ]:
mean_list = []
std_dev_list = []
n_list = []

for los_vals in los_vals_list:
    los_vals = np.array(los_vals)
    mean_list.append(np.mean(los_vals))
    std_dev_list.append(np.std(los_vals))

    n_list.append(len(los_vals))

print(mean_list, std_dev_list, n_list)

In [ ]:
# Check critical values from table/ppf function
alpha = 0.05
t_critical_list = []

for n in n_list:
    dof = n - 1
    t_critical_list.append(t.ppf(1-alpha/2, dof))

print(t_critical_list)

In [ ]:
# Calculate confidence interval
# H_val = t_critical * std_dev * (np.sqrt(1 + (1/n)))

H_val_list = []
lower_bound = []
upper_bound = []

for index, t_critical in enumerate(t_critical_list):
    H_val = t_critical * std_dev_list[index] * (1/np.sqrt(n_list[index]))
    H_val_list.append(H_val)
    lower_bound.append(mean_list[index] - H_val)
    upper_bound.append(mean_list[index] + H_val)

print(lower_bound, upper_bound, H_val_list)
# print([mean - lower_bound], [upper_bound - mean])

In [ ]:
# Across rep data
var_list = [a**2 for a in std_dev_list]

a_mean = sum(mean_list)/5

a_var = 0
for means in mean_list:
    a_var += (means - a_mean) ** 2

a_var = a_var/4
a_std_dev = np.sqrt(a_var)

a_H_val = t_critical_list[0] * a_std_dev * (1/np.sqrt(5))

print(a_mean, a_std_dev, a_H_val, var_list, a_var, a_std_dev)

In [ ]:
a_lower_bound = a_mean - a_H_val
a_upper_bound = a_mean + a_H_val
print(a_lower_bound, a_upper_bound)

In [ ]:
# Plot boxplot for confidence intervals
# fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 8))
interval_list = []
for index, mean in enumerate(mean_list):
    interval_list.append([lower_bound[index], mean, upper_bound[index]])

for index, interval in enumerate(interval_list):
    plt.boxplot(interval, medianprops=dict(color="red"), positions=[index+1])
    # plt.plot((index, index, index), interval)
    # plt.axhline(y=interval[1], color='orange', linestyle='--')
    
plt.show()

## Simulation System 4

Please note, simulation system 4 is presented as system 5 in the report. 

In [6]:
# Read simulation results as dataframe
df41 = pd.read_csv("../results/simulation_results_system_4_1.csv", sep=r",(?![^\[]*\])")
df42 = pd.read_csv("../results/simulation_results_system_4_2.csv", sep=r",(?![^\[]*\])")
df43 = pd.read_csv("../results/simulation_results_system_4_3.csv", sep=r",(?![^\[]*\])")
df44 = pd.read_csv("../results/simulation_results_system_4_4.csv", sep=r",(?![^\[]*\])")
df45 = pd.read_csv("../results/simulation_results_system_4_5.csv", sep=r",(?![^\[]*\])")
df_list = [df41, df42, df43, df44, df45]

/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6537/2783056028.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df41 = pd.read_csv("../results/simulation_results_system_4_1.csv", sep=r",(?![^\[]*\])")
/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6537/2783056028.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df42 = pd.read_csv("../results/simulation_results_system_4_2.csv", sep=r",(?![^\[]*\])")
/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6537/2783056028.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex 

In [7]:
for df in df_list:
    df.columns = df.columns.str.lstrip()

In [8]:
# Read LOS data from dataframe df
# LOS values of only treated patients is considered
# Treated patients are the ones that complete their run (arrival to departure) in the ER

los_vals_1 = []
los_vals_2 = []
los_vals_3 = []
los_vals_4 = []
los_vals_5 = []

for items in df41["LOS"]:
    if items > 0:
        los_vals_1.append(items/60)

for items in df42["LOS"]:
    if items > 0:
        los_vals_2.append(items/60)
        
for items in df43["LOS"]:
    if items > 0:
        los_vals_3.append(items/60)
        
for items in df44["LOS"]:
    if items > 0:
        los_vals_4.append(items/60)
        
for items in df45["LOS"]:
    if items > 0:
        los_vals_5.append(items/60)

los_vals_list = [los_vals_1, los_vals_2, los_vals_3, los_vals_4, los_vals_5]
print(min(los_vals_1), max(los_vals_1))

0.10359808058747148 4.0180206389295865


In [11]:
mean_list = []
std_dev_list = []
n_list = []

for los_vals in los_vals_list:
    los_vals = np.array(los_vals)
    mean_list.append(np.mean(los_vals))
    std_dev_list.append(np.std(los_vals))

    n_list.append(len(los_vals))

print(mean_list, std_dev_list, n_list)

[1.331521016802784, 1.3281774664505903, 1.324874721653778, 1.3252920040854128, 1.3281790876561017] [0.6660473678764711, 0.6568311416243993, 0.6540822160054085, 0.6587849430909237, 0.6619598354775071] [98760, 97730, 98064, 98493, 98454]


In [ ]:
# Check critical values from table/ppf function
alpha = 0.05
t_critical_list = []

for n in n_list:
    dof = n - 1
    t_critical_list.append(t.ppf(1-alpha/2, dof))

print(t_critical_list)

In [ ]:
# Calculate confidence interval
# H_val = t_critical * std_dev * (np.sqrt(1 + (1/n)))

H_val_list = []
lower_bound = []
upper_bound = []

for index, t_critical in enumerate(t_critical_list):
    H_val = t_critical * std_dev_list[index] * (1/np.sqrt(n_list[index]))
    H_val_list.append(H_val)
    lower_bound.append(mean_list[index] - H_val)
    upper_bound.append(mean_list[index] + H_val)

print(lower_bound, upper_bound, H_val_list)
# print([mean - lower_bound], [upper_bound - mean])

In [ ]:
# Across rep data
var_list = [a**2 for a in std_dev_list]

a_mean = sum(mean_list)/5

a_var = 0
for means in mean_list:
    a_var += (means - a_mean) ** 2

a_var = a_var/4
a_std_dev = np.sqrt(a_var)

a_H_val = t_critical_list[0] * a_std_dev * (1/np.sqrt(5))

print(a_mean, a_std_dev, a_H_val, var_list, a_var, a_std_dev)

In [ ]:
a_lower_bound = a_mean - a_H_val
a_upper_bound = a_mean + a_H_val
print(a_lower_bound, a_upper_bound)

In [ ]:
# Plot boxplot for confidence intervals
# fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 8))
interval_list = []
for index, mean in enumerate(mean_list):
    interval_list.append([lower_bound[index], mean, upper_bound[index]])

for index, interval in enumerate(interval_list):
    plt.boxplot(interval, medianprops=dict(color="red"), positions=[index+1])
    # plt.plot((index, index, index), interval)
    # plt.axhline(y=interval[1], color='orange', linestyle='--')
    
plt.show()

## Simulation System 5


Please note, this simulation system is presented as system 4 in the report. 

In [2]:
# Read simulation results as dataframe
df41 = pd.read_csv("../results/simulation_results_system_5_1.csv", sep=r",(?![^\[]*\])")
df42 = pd.read_csv("../results/simulation_results_system_5_2.csv", sep=r",(?![^\[]*\])")
df43 = pd.read_csv("../results/simulation_results_system_5_3.csv", sep=r",(?![^\[]*\])")
df44 = pd.read_csv("../results/simulation_results_system_5_4.csv", sep=r",(?![^\[]*\])")
df45 = pd.read_csv("../results/simulation_results_system_5_5.csv", sep=r",(?![^\[]*\])")
df_list = [df41, df42, df43, df44, df45]

/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6537/495048192.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df41 = pd.read_csv("../results/simulation_results_system_5_1.csv", sep=r",(?![^\[]*\])")
/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6537/495048192.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df42 = pd.read_csv("../results/simulation_results_system_5_2.csv", sep=r",(?![^\[]*\])")
/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6537/495048192.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex sep

In [3]:
for df in df_list:
    df.columns = df.columns.str.lstrip()

In [4]:
# Read LOS data from dataframe df
# LOS values of only treated patients is considered
# Treated patients are the ones that complete their run (arrival to departure) in the ER

los_vals_1 = []
los_vals_2 = []
los_vals_3 = []
los_vals_4 = []
los_vals_5 = []

for items in df41["LOS"]:
    if items > 0:
        los_vals_1.append(items/60)

for items in df42["LOS"]:
    if items > 0:
        los_vals_2.append(items/60)
        
for items in df43["LOS"]:
    if items > 0:
        los_vals_3.append(items/60)
        
for items in df44["LOS"]:
    if items > 0:
        los_vals_4.append(items/60)
        
for items in df45["LOS"]:
    if items > 0:
        los_vals_5.append(items/60)

los_vals_list = [los_vals_1, los_vals_2, los_vals_3, los_vals_4, los_vals_5]
print(min(los_vals_1), max(los_vals_1))

0.13884777294015294 22.737922349409406


In [5]:
mean_list = []
std_dev_list = []
n_list = []

for los_vals in los_vals_list:
    los_vals = np.array(los_vals[100:90100])
    mean_list.append(np.mean(los_vals))
    std_dev_list.append(np.std(los_vals))

    n_list.append(len(los_vals))

print(mean_list, std_dev_list, n_list)

[4.252080105808182, 4.223080759046099, 3.6955200636221366, 4.706032683692952, 4.193880159542005] [3.3901592317724254, 3.0665125984007244, 2.7731345734386332, 3.5831767741357483, 3.1789715878181934] [1525, 1701, 1487, 2057, 1699]


In [ ]:
# Check critical values from table/ppf function
alpha = 0.05
t_critical_list = []

for n in n_list:
    dof = n - 1
    t_critical_list.append(t.ppf(1-alpha/2, dof))

print(t_critical_list)

In [ ]:
# Calculate confidence interval
# H_val = t_critical * std_dev * (np.sqrt(1 + (1/n)))

H_val_list = []
lower_bound = []
upper_bound = []

for index, t_critical in enumerate(t_critical_list):
    H_val = t_critical * std_dev_list[index] * (1/np.sqrt(n_list[index]))
    H_val_list.append(H_val)
    lower_bound.append(mean_list[index] - H_val)
    upper_bound.append(mean_list[index] + H_val)

print(lower_bound, upper_bound, H_val_list)
# print([mean - lower_bound], [upper_bound - mean])

In [ ]:
# Across rep data
var_list = [a**2 for a in std_dev_list]

a_mean = sum(mean_list)/5

a_var = 0
for means in mean_list:
    a_var += (means - a_mean) ** 2

a_var = a_var/4
a_std_dev = np.sqrt(a_var)

a_H_val = t_critical_list[0] * a_std_dev * (1/np.sqrt(5))

print(a_mean, a_std_dev, a_H_val, var_list, a_var, a_std_dev)

In [ ]:
a_lower_bound = a_mean - a_H_val
a_upper_bound = a_mean + a_H_val
print(a_lower_bound, a_upper_bound)

In [ ]:
# Plot boxplot for confidence intervals
# fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 8))
interval_list = []
for index, mean in enumerate(mean_list):
    interval_list.append([lower_bound[index], mean, upper_bound[index]])

for index, interval in enumerate(interval_list):
    plt.boxplot(interval, medianprops=dict(color="red"), positions=[index+1])
    # plt.plot((index, index, index), interval)
    # plt.axhline(y=interval[1], color='orange', linestyle='--')
    
plt.show()